In [1]:
using GaussianProcesses
using GaussianProcesses: update_mll_and_dmll!, update_cK!
using BenchmarkTools
using DataFrames
using JLD

const d = 10        # Input observation dimension
const nt = 3000     # Number of training points

kerns = Dict(
    "se" => SEIso(0.0,0.0),
    "mat12" => Mat12Iso(0.0,0.0),
    "rq" => RQIso(0.0,0.0,0.0),
    "se+rq" => SEIso(0.0,0.0) + RQIso(0.0,0.0,0.0),
    "se*rq" => SEIso(0.0,0.0) * RQIso(0.0,0.0,0.0),
    "se+se2+rq" => SEIso(0.0,0.0) + SEIso(0.5,0.5) + RQIso(0.0,0.0,0.0),
    "(se+se2)*rq" => (SEIso(0.0,0.0) + SEIso(0.5,0.5)) * RQIso(0.0,0.0,0.0),
    "mask(se, [1])" => Masked(SEIso(0.0,0.0), [1]),
    "mask(se, [1])+mask(rq, [2:10])" =>  Masked(SEIso(0.0,0.0), [1]) +  Masked(RQIso(0.0,0.0,0.0), collect(2:10)),
    "fix(se, σ)" => fix(SEIso(0.0,0.0), :lσ)
    )
    
function benchmark_kernel(group, kern)
    srand(1)
    X = randn(d, nt) # Training data
    Y = randn(nt)
    buf1=Array{Float64}(nt,nt)
    buf2=Array{Float64}(nt,nt)
    gp = GP(X, Y, MeanConst(0.0), kern, log(1.0))
    group["cK"] = @benchmarkable update_cK!($gp)
    group["mll_and_dmll"] = @benchmarkable update_mll_and_dmll!($gp, $buf1, $buf2)
end

SUITE = BenchmarkGroup()

for (label, k) in kerns
    SUITE[label] = BenchmarkGroup([label])
    benchmark_kernel(SUITE[label], k)
end
;

In [2]:
results = run(SUITE, verbose=false, seconds=1000, samples=10, evals=1)
;

In [3]:
knames = sort(collect(keys(kerns)))
times = [time(results[k]["mll_and_dmll"])/10^6 for k in knames]

df = DataFrame(kernel = knames, times=times)
print(df)

10×2 DataFrames.DataFrame
│ Row │ kernel                         │ times   │
├─────┼────────────────────────────────┼─────────┤
│ 1   │ (se+se2)*rq                    │ 4445.29 │
│ 2   │ fix(se, σ)                     │ 796.74  │
│ 3   │ mask(se, [1])                  │ 928.841 │
│ 4   │ mask(se, [1])+mask(rq, [2:10]) │ 1817.31 │
│ 5   │ mat12                          │ 920.966 │
│ 6   │ rq                             │ 1359.79 │
│ 7   │ se                             │ 895.128 │
│ 8   │ se*rq                          │ 2693.16 │
│ 9   │ se+rq                          │ 1737.77 │
│ 10  │ se+se2+rq                      │ 2099.03 │

In [4]:
writetable("bench_results/GaussianProcesses_jl.csv", df, header=false)

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] #writetable#223(::Bool, ::Char, ::Char, ::String, ::Bool, ::Function, ::String, ::DataFrames.DataFrame) at /Users/imolk/Library/Julia/packages_experimental/v0.6/DataFrames/src/deprecated.jl:69
 [3] (::DataFrames.#kw##writetable)(::Array{Any,1}, ::DataFrames.#writetable, ::String, ::DataFrames.DataFrame) at ./<missing>:0
 [4] include_string(::String, ::String) at /Users/imolk/bin/julia_v0.6.2_binary/lib/julia/sys.dylib:?
 [5] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/imolk/Library/Julia/packages_experimental/v0.6/IJulia/src/execute_request.jl:158
 [6] (::Compat.#inner#18{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /Users/imolk/Library/Julia/packages_experimental/v0.6/Compat/src/Compat.jl:386
 [7] eventloop(::ZMQ.Socket) at /Users/imolk/Library/Julia/packages_experimental/v0.6/IJulia/src/eventloop.jl:8
 [8] (::IJulia.##14#17)() at ./task.jl:335
while loading In[4], in expressi

# Profile `update_mll_and_dmll!`

In [5]:
srand(1)
X = randn(d, nt) # Training data
Y = randn(nt)
XY_df = DataFrame(X')
XY_df[:Y] = Y
XY_df
writetable("simdata.csv", XY_df, header=true)

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] #writetable#223(::Bool, ::Char, ::Char, ::String, ::Bool, ::Function, ::String, ::DataFrames.DataFrame) at /Users/imolk/Library/Julia/packages_experimental/v0.6/DataFrames/src/deprecated.jl:69
 [3] (::DataFrames.#kw##writetable)(::Array{Any,1}, ::DataFrames.#writetable, ::String, ::DataFrames.DataFrame) at ./<missing>:0
 [4] include_string(::String, ::String) at /Users/imolk/bin/julia_v0.6.2_binary/lib/julia/sys.dylib:?
 [5] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/imolk/Library/Julia/packages_experimental/v0.6/IJulia/src/execute_request.jl:158
 [6] (::Compat.#inner#18{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /Users/imolk/Library/Julia/packages_experimental/v0.6/Compat/src/Compat.jl:386
 [7] eventloop(::ZMQ.Socket) at /Users/imolk/Library/Julia/packages_experimental/v0.6/IJulia/src/eventloop.jl:8
 [8] (::IJulia.##14#17)() at ./task.jl:335
while loading In[5], in expressi

In [6]:
buf1=Array{Float64}(nt,nt)
buf2=Array{Float64}(nt,nt)
gp = GPE(X, Y, MeanConst(0.0), kerns["se"], log(1.0))
update_mll_and_dmll!(gp, buf1, buf2)
gp.mll, gp.dmll # SE kernel

(-4536.259909444878, [-689.634, -15.7312, 71.1964, -667.268])

In [7]:
buf1=Array{Float64}(nt,nt)
buf2=Array{Float64}(nt,nt)
gp = GPE(X, Y, MeanConst(0.0), kerns["rq"], log(1.0))
Profile.clear()
update_mll_and_dmll!(gp, buf1, buf2)
@profile update_mll_and_dmll!(gp, buf1, buf2)
@profile update_mll_and_dmll!(gp, buf1, buf2)

In [8]:
Profile.print(mincount=10)

566 ./task.jl:335; (::IJulia.##14#17)()
 566 ...Julia/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
  566 .../Compat/src/Compat.jl:386; (::Compat.#inner#18{Array{Any,1}...
   565 ...rc/execute_request.jl:158; execute_request(::ZMQ.Socket, ::...
    565 ...y/lib/julia/sys.dylib:?; include_string(::String, ::String)
     564 ./<missing>:?; anonymous
      564 ./profile.jl:23; macro expansion
       564 ...rocesses/src/GPE.jl:209; update_mll_and_dmll!(::Gaussi...
        130 ...rocesses/src/GPE.jl:209; #update_mll_and_dmll!#95(::Ar...
         130 ...ocesses/src/GPE.jl:156; #update_mll!#93(::Bool, ::Bo...
          106 ...ocesses/src/GPE.jl:141; update_cK!(::GaussianProces...
           106 ...nels/stationary.jl:89; cov!(::Array{Float64,2}, ::...
            95 ...nels/stationary.jl:82; cov!(::Array{Float64,2}, ::...
             64 ...rnels/distance.jl:39; distij
              64 ...rnels/distance.jl:24; _SqEuclidean_ij
               11 ./simdloop.jl:71; macro expansion
               40 